In [ ]:
import pandas as pd
import numpy as np
from sqlalchemy import create_engine
from pandas.io import sql

import os

In [ ]:
# ARIMA를 위한 월별 데이터 추출
total_2015DF=pd.DataFrame()
for filename in os.listdir(""):
    print(filename)
    data=pd.read_csv(''%filename, encoding='utf-8')
    
    del data['측정소명']
    del data['PM25']
    del data['주소']

    data[data.columns[0]]=data[data.columns[0]].str[:2]

    data['측정일시']=data.측정일시.apply(str)
    data['측정일시']=data.측정일시.str[:6]
    test_data=pd.to_datetime(data['측정일시'], format="%Y%m")
    data['측정일시']=test_data.apply(lambda x: x.strftime('%Y-%m'))

    for i in range(4,len(data.columns)-1):
        data=data[data[data.columns[i]]>0]

    data.columns=['Location','Date','SO2','CO','O3','NO2','PM10']
    data=data.groupby(['Location','Date'])['SO2','CO','O3','NO2','PM10'].mean()

    for i in range(0,len(data.columns)):
        data[data.columns[i]]=np.around(data[data.columns[i]],3)
        
    total_2015DF=total_2015DF.append(data)

total_2015DF=total_2015DF.reset_index()

engine = create_engine('')
cnn = engine.connect()

dbData=sql.read_sql("", cnn)

del dbData['idx']
total=dbData.append(total_2015DF)
total=total.sort(['Date'], ascending=[True])

In [ ]:
#CSV save
total.to_csv('월별Total.csv')

In [ ]:
''' 2016년도 1~3월까지 '''
total_2016DF=pd.DataFrame()
for filename in os.listdir(""):
    data=pd.read_csv(''%filename, encoding='utf-8')
    del data['측정소코드']
    del data['PM2.5']
    del data['측정소명']

    data.columns = ['Date','Location','SO2','CO','O3','NO2','PM10']
    data[data.columns[1]]=data[data.columns[1]].str[:2]
    data['Date']=data.Date.apply(str)
    data['Date']=data.Date.str[:7]
    test_data=pd.to_datetime(data['Date'], format="%Y-%m")
    data['Date']=test_data.apply(lambda x: x.strftime('%Y-%m'))
    data=np.round(data.groupby(['Date','Location'])['SO2','CO','O3','NO2','PM10'].mean(),3)
    data=data.reset_index()
    
    total_2016DF=total_2016DF.append(data)

'''
2005~2015년까지의 데이터와 2016년도의 데이터를 합침
'''
result=total.append(total_2016DF)
result=pd.DataFrame(result,columns=['Date','Location','NO2','SO2','CO','PM10'])


In [ ]:
result.to_csv("월별Total_2005_01to2016_03.csv")

In [ ]:
total_2016DF.to_csv("201603월별.csv")